# Linear Regression and Data Visualization with Seaborn

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('data/cost_revenue_dirty.csv')

## Data Exploration and Cleaning

In [29]:
print(f'The data have {df.shape[0]} rows and {df.shape[1]} columns')
print(f'The data have {df.isna().sum().sum()} NaN values')
print(f'The data have {df.duplicated().sum()} duplicated rows')
print(f'Type of columns:\n{df.dtypes}')

The data have 5391 rows and 6 columns
The data have 0 NaN values
The data have 0 duplicated rows
Type of columns:
Rank                              int64
Release_Date             datetime64[ns]
Movie_Title                      object
USD_Production_Budget            object
USD_Worldwide_Gross              object
USD_Domestic_Gross               object
dtype: object


In [16]:
# Removing $ and , from the columns
df['USD_Production_Budget'] = df['USD_Production_Budget'].str.replace('$', '').str.replace(',', '')
df['USD_Worldwide_Gross'] = df['USD_Worldwide_Gross'].str.replace('$', '').str.replace(',', '')
df['USD_Domestic_Gross'] = df['USD_Domestic_Gross'].str.replace('$', '').str.replace(',', '')

In [24]:
df.Release_Date = pd.to_datetime(df.Release_Date)

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
